In [1]:
import torch
import sys
import os
import importlib
import torch
import numpy
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict
import pickle
from asnorm import *

In [2]:
# make cohort set

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [4]:
sys.path.append('/workspace/GREAT_ASV_system/train_dist')

In [5]:
# from DatasetLoader import loadWAV

In [6]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNN').__getattribute__('MainModel')

In [7]:
# EPACA-TDNN
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNN.py, Embedding size is 192, Channels 1024, Spec_aug False.


In [8]:
model_path = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/multi_gpu_epaca_tdnn_soxaug/model/model000000034.model'

In [9]:
train_list = '/workspace/DATASET/server9_ssd/voxceleb/vox2_trainlist.txt'
train_path = '/workspace/DATASET/server9_ssd/voxceleb'
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'
score_file = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/multi_gpu_epaca_tdnn_soxaug/result/model000000034.vox1osc'
out_path_1 = 'as1.txt'
out_path_2 = 'as2.txt'

In [10]:
S.cuda()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [11]:
loaded_state = torch.load(model_path, map_location="cuda:0")

In [12]:
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name
    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state[origname].size()))
        continue

    self_state[name].copy_(param)

#__L__.W is not in the model.


In [13]:
S.eval()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [14]:
def loadWAV(filename, max_frames):

    # Maximum audio length
    max_audio = max_frames * 160 + 240

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)

    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]

    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat;

In [15]:
with open(train_list, 'r') as f:
    train_list_list = f.readlines()

In [16]:
len(train_list_list)

1092009

In [17]:
cohort_spk_dict = {}
for count, line in enumerate(train_list_list):
    label, wavline = line[:-1].split(' ')
    wavline = os.path.join(train_path, wavline)
    raw_inp = loadWAV(wavline, max_frames=0)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu()
    if label not in cohort_spk_dict.keys():
        cohort_spk_dict[label] = ref_feat
    else:
        cohort_spk_dict[label] = torch.cat([cohort_spk_dict[label], ref_feat], dim=0)
    if ((count+1) % 1000) == 0:
        print((count+1)//1000, end='\r')

In [18]:
cohort_spk_dict_mean = {}
for i in cohort_spk_dict:
    cohort_spk_dict_mean[i] = torch.mean(cohort_spk_dict[i], dim=0, keepdim=False)

In [19]:
cohort_spk_dict_mean_nm = {}
for i in cohort_spk_dict_mean:
    cohort_spk_dict_mean_nm[i] = F.normalize(cohort_spk_dict_mean[i], p=2, dim=0)

In [20]:
# extract vox-o

In [21]:
listfilename = test_list
files = []
with open(listfilename) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split()

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [22]:
len(setfiles)

4708

In [23]:
test_dict = {}

In [24]:
for count, line in enumerate(setfiles):
    wavline = line
    wavline = os.path.join(train_path, wavline)
    raw_inp = loadWAV(wavline, max_frames=0)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu()

    test_dict[line] = ref_feat
    
    if ((count+1) % 1000) == 0:
        print((count+1)//1000, end='\r')

In [25]:
len(test_dict)

4708

In [26]:
test_dict_nm = {}
for i in test_dict:
    test_dict_nm[i] = F.normalize(test_dict[i].squeeze(0), p=2, dim=0)

In [27]:
### optional
# import pickle
# with open('tmp_chort_dict', 'wb') as f:
#     pickle.dump(cohort_spk_dict, f)
# import pickle
# with open('tmp_test_dict', 'wb') as f:
#     pickle.dump(test_dict, f)

In [35]:
count = 0
cohort2test = {}
for i in cohort_spk_dict_mean_nm:
    cohort_emb = cohort_spk_dict_mean_nm[i]
    for j in test_dict_nm:
        test_emb = test_dict_nm[j]
        score = F.cosine_similarity(cohort_emb, test_emb, dim=0).cpu().numpy()
        cohort2test[i+" "+j] = score

        count += 1
        if (count % 100000) == 0:
            print(count // 100000, end='\r')
                
print(count)

28219752


In [29]:
# input score_file; enroll_audio; test_audio; cohort_audio

In [36]:
as_norm_1(score_file, out_path_1, cohort2test, cohort2test, top_num=2000, hold_name=True)

Score norm operation start, method: Adaptive_score_norm_type_1
calculate all statistics
Scoring...
Adaptive_score_norm_type_1 is finished


In [37]:
as_norm_2(score_file, out_path_2, cohort2test, cohort2test, top_num=2000, hold_name=True)

Score norm operation start, method: Adaptive_score_norm_type_2
Scoring...
Adaptive_score_norm_type_2 is finished


In [ ]:
# get asnormed score_file & scoring